# STC Jawwy

In [1]:
"""
Here we install libraries that are not installed by default 
Example:  pyslsb
Feel free to add any library you are planning to use.
"""
!pip install pyxlsb
!pip install torch
!pip install tqdm


[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
# we import Visualization libraries 
# you can ignore and use any other graphing libraries 
import matplotlib.pyplot as plt # a comprehensive library for creating static, animated, and interactive visualizations
import plotly #a graphing library makes interactive, publication-quality graphs. Examples of how to make line plots, scatter plots, area charts, bar charts, error bars, box plots, histograms, heatmaps, subplots, multiple-axes, polar charts, and bubble charts.
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import torch
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

import pandas as pd
import pyxlsb
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import torch
from torch.autograd import Variable
from tqdm import tqdm
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import pandas as pd
import pyxlsb
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import torch
from torch.autograd import Variable
from tqdm import tqdm
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from sklearn.cluster import KMeans

In [3]:
# Import the required libraries  
"""
Please feel free to import any required libraries as per your needs
"""
import pandas as pd     # provides high-performance, easy to use structures and data analysis tools
import pyxlsb           # Excel extention to read xlsb files (the input file)
import numpy as np      # provides fast mathematical computation on arrays and matrices

# Jawwy dataset
The dataset consists of details about each customer and the movies and/or tv shows watched in addition to the genre. 

You are required to work on task three to build a recommendation engine for our platform to Recommend movies to usesrs that they might be interested in¶


In [4]:
dataframe = pd.read_excel("C:\\Users\\PC\\Downloads\\stc TV Data Set_T3.xlsx",index_col=0)
# Please make a copy of dataset if you are going to work directly and make changes on the dataset
# you can use   df=dataframe.copy()

In [46]:
dataframe['user_id_maped'] = dataframe['user_id_maped'].astype(int)
dataframe['program_name'] = dataframe['program_name'].astype(str)
dataframe['rating'] = dataframe['rating'].astype(float)


In [25]:
# check the data shape 
print("Data shape:", dataframe.shape)

Data shape: (1048575, 5)


In [6]:
# display the first 5 rows 
dataframe.head()

,user_id_maped,program_name,rating,date_,program_genre
0,26138,100 treets,1,2017-05-27,Drama
1,7946,Moana,1,2017-05-21,Animation
2,7418,The Mermaid Princess,1,2017-08-10,Animation
3,19307,The Mermaid Princess,2,2017-07-26,Animation
4,15860,Churchill,2,2017-07-07,Biography


In [7]:
# describe the numeric values in the dataset
dataframe.describe()

,user_id_maped,rating,date_
count,1.048575e+06,1.048575e+06,1048575
mean,1.709266e+04,2.497283e+00,2017-10-04 00:23:20.346183936
min,1.000000e+00,1.000000e+00,2017-03-14 00:00:00
25%,8.253000e+03,1.000000e+00,2017-06-10 00:00:00
50%,1.714900e+04,2.000000e+00,2017-10-14 00:00:00
75%,2.566500e+04,3.000000e+00,2018-01-21 00:00:00
max,3.428000e+04,4.000000e+00,2018-04-30 00:00:00
std,1.003513e+04,1.119837e+00,NaN


In [8]:
# check if any column has null value in the dataset
dataframe.isnull().any()

user_id_maped    False
program_name     False
rating           False
date_            False
program_genre    False
dtype: bool

In [26]:
movie_names = dataframe.set_index('program_name')['program_genre'].to_dict()
n_users = len(dataframe.user_id_maped.unique())
n_items = len(dataframe.program_name.unique())

print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users * n_items, 'elements')
print("Number of ratings:", len(dataframe.rating))
print("Matrix fill percentage:", len(dataframe.rating) / (n_users * n_items) * 100, '%')
print("We have an incredibly sparse matrix to work with here.")

Number of unique users: 11578
Number of unique movies: 8013
The full rating matrix will have: 92774514 elements
Number of ratings: 1048575
Matrix fill percentage: 1.1302403588985657 %
We have an incredibly sparse matrix to work with here.


In [48]:
class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, n_factors)
        self.item_factors = torch.nn.Embedding(n_items, n_factors)
        self.user_factors.weight.data.uniform_(0, 0.5)
        self.item_factors.weight.data.uniform_(0, 0.5)

    def forward(self, data):
        users, items = data[:, 0], data[:, 1]
        return (self.user_factors(users) * self.item_factors(items)).sum(1)

    def predict(self, user, item):
        return self.forward(torch.tensor([[user, item]]))

In [64]:
class Loader(Dataset):
    def __init__(self):
        self.rating = dataframe.copy()

        users = self.rating.user_id_maped.unique()
        movies = self.rating.program_name.unique()
        self.user_id_maped2idx = {o: i for i, o in enumerate(users)}
        self.program_name2idx = {o: i for i, o in enumerate(movies)}
        self.idx2user_id_mapped = {i: o for o, i in self.user_id_maped2idx.items()}
        self.idx2program_name = {i: o for o, i in self.program_name2idx.items()}
        self.rating.program_name = self.rating.program_name.apply(lambda x: self.program_name2idx[x])
        self.rating.user_id_maped = self.rating.user_id_maped.apply(lambda x: self.user_id_maped2idx[x])
        self.x = self.rating.drop(['rating', 'date_'], axis=1).values
        self.y = self.rating['rating'].values
        self.x, self.y = torch.tensor(self.x, dtype=torch.long), torch.tensor(self.y, dtype=torch.float)

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return len(self.rating)

In [65]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: False
MatrixFactorization(
  (user_factors): Embedding(11578, 8)
  (item_factors): Embedding(8013, 8)
)
user_factors.weight tensor([[0.1656, 0.2736, 0.2011,  ..., 0.0439, 0.3111, 0.1968],
        [0.4478, 0.2388, 0.3345,  ..., 0.2990, 0.0273, 0.4866],
        [0.3393, 0.0929, 0.4088,  ..., 0.2256, 0.3237, 0.2064],
        ...,
        [0.3823, 0.0849, 0.1879,  ..., 0.2561, 0.0446, 0.3756],
        [0.2007, 0.3883, 0.3754,  ..., 0.1518, 0.0206, 0.2575],
        [0.0050, 0.1684, 0.4239,  ..., 0.2975, 0.0264, 0.0691]])
item_factors.weight tensor([[0.2057, 0.0438, 0.1633,  ..., 0.2053, 0.0178, 0.1045],
        [0.2819, 0.1386, 0.3152,  ..., 0.2658, 0.4780, 0.1589],
        [0.0957, 0.1816, 0.1573,  ..., 0.3674, 0.1626, 0.3018],
        ...,
        [0.0079, 0.3572, 0.4313,  ..., 0.1431, 0.2459, 0.4563],
        [0.1263, 0.2014, 0.1538,  ..., 0.2828, 0.1136, 0.0448],
        [0.2809, 0.3787, 0.3826,  ..., 0.1495, 0.1774, 0.1201]])


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [66]:
for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
         if cuda:
            x, y = x.cuda(), y.cuda()
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
    print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))

  0%|          | 0/128 [00:00<?, ?it/s]


NameError: name 'train_loader' is not defined

In [62]:
# Extract latent factors for users and items
user_factors = model.user_factors.weight.data.cpu().numpy()
item_factors = model.item_factors.weight.data.cpu().numpy()

# Perform KMeans clustering
kmeans = KMeans(n_clusters=10, random_state=0).fit(item_factors)

# Output movie clusters
for cluster in range(10):
    print("Cluster #{}".format(cluster))
    movs = []
    for movidx in np.where(kmeans.labels_ == cluster)[0]:
        movid = train_set.idx2program_name[movidx]
        rat_count = dataframe.loc[dataframe['program_name'] == movid].count()[0]
        movs.append((movid, rat_count))
    for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
        print("\t", mov[0]) 

Cluster #0


NameError: name 'train_set' is not defined

In [53]:
c = 0
uw = 0
iw = 0 
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        #print('param_data', param_data)

user_factors.weight tensor([[0.4371, 0.4410, 0.3213,  ..., 0.1491, 0.2693, 0.4126],
        [0.0047, 0.1863, 0.4835,  ..., 0.0581, 0.3508, 0.2019],
        [0.0642, 0.2843, 0.0107,  ..., 0.0960, 0.4194, 0.3409],
        ...,
        [0.3131, 0.3458, 0.3717,  ..., 0.2334, 0.1022, 0.4189],
        [0.4103, 0.1695, 0.3901,  ..., 0.1323, 0.3401, 0.4189],
        [0.0349, 0.0783, 0.2460,  ..., 0.0637, 0.0681, 0.4956]])
item_factors.weight tensor([[0.4753, 0.3165, 0.0416,  ..., 0.1856, 0.3382, 0.2976],
        [0.0721, 0.2021, 0.4602,  ..., 0.3230, 0.0466, 0.3389],
        [0.2283, 0.3089, 0.4410,  ..., 0.0132, 0.4025, 0.0441],
        ...,
        [0.0280, 0.2462, 0.4546,  ..., 0.3964, 0.0558, 0.0810],
        [0.0857, 0.4785, 0.0915,  ..., 0.2165, 0.1228, 0.1424],
        [0.1184, 0.3900, 0.3048,  ..., 0.3014, 0.4571, 0.3172]])


In [54]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()


In [55]:
len(trained_movie_embeddings) # unique movie factor weights


8013

In [57]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

In [58]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
  print("Cluster #{}".format(cluster))
  movs = []
  for movidx in np.where(kmeans.labels_ == cluster)[0]:
    movid = train_set.idx2movieid[movidx]
    rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
    movs.append((movie_names[movid], rat_count))
  for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t", mov[0])

Cluster #0


NameError: name 'train_set' is not defined

In [67]:
def get_recommendations(movie_name, top_n=5):
    # Convert the movie name to its index
    movie_idx = train_set.program_name2idx[movie_name]
    
    # Get the embedding vector for the movie
    movie_vec = model.item_factors(torch.tensor([movie_idx]).cuda()).detach().cpu().numpy()
    
    # Calculate similarity scores with all users
    scores = torch.matmul(model.user_factors.weight, torch.tensor(movie_vec).T.cuda()).squeeze()
    
    # Get top N users with the highest scores
    _, top_users = torch.topk(scores, top_n)
    
    # Convert the indices back to user ids
    return [train_set.idx2user_id_mapped[user.item()] for user in top_users]

# Example usage
recommendations = get_recommendations("Moana", top_n=5)
print("Top 5 users who watched 'Moana':", recommendations)


NameError: name 'train_set' is not defined